In [1]:
from pymongo import MongoClient

In [2]:
import pandas as pd

In [3]:
%matplotlib inline

In [4]:
def return_collection_fields(db, collection_name):
    return pd.DataFrame([_['attributes'] for _ in db[collection_name].find()])

In [5]:
mongo_client = MongoClient(host='35.156.12.147')
db = mongo_client['raw_data']

In [6]:
vilnius_stats_df = return_collection_fields(db, 'vilnius_stats')
vilnius_stats_df = vilnius_stats_df.drop_duplicates(subset=['data'])
vilnius_stats_df = vilnius_stats_df[['data', 'susirge', 'mire', 'dienos_susirg']]
vilnius_stats_df['date'] = pd.to_datetime(vilnius_stats_df['data'], unit='ms').dt.round('D')
vilnius_stats_df = vilnius_stats_df.drop('data', axis=1)

In [7]:
personel_df = return_collection_fields(db, 'personel')
# personel_df = personel[['Laiko_žymė', 'Įstaigos_pavadinimas', 'Medikų_kiekis__VISO_', 'Karantinuota__VISO']]
personel_df['Laiko_žymė'] = pd.to_datetime(personel_df['Laiko_žymė'], unit='ms').dt.round('D')

In [8]:
inventory_df = return_collection_fields(db, 'inventory')
inventory_df['Laiko_žymė'] = pd.to_datetime(inventory_df['Laiko_žymė'], unit='ms').dt.round('D')

In [9]:
personel_aggregated = personel_df\
    .groupby(['Laiko_žymė', 'Įstaigos_pavadinimas'])[personel_df.columns[3:-3]]\
    .min()\
    .reset_index()\
    .groupby('Laiko_žymė')[personel_df.columns[3:-3]]\
    .sum()\
    .reset_index()\
    .rename({'Laiko_žymė': 'date'}, axis=1)

personel_aggregated['quarantined_pct'] = personel_aggregated['Karantinuota__VISO']/personel_aggregated['Medikų_kiekis__VISO_']
personel_aggregated['medics_total'] = personel_aggregated[['Medikai_reanimatologai__VISO', 'Medikų_kiekis__VISO_', 'Apmokyti_medikai', 'Atsarga']].sum(axis=1)

In [10]:
personel_aggregated.to_clipboard()

In [11]:
inventory_aggregated = inventory_df\
    .groupby(['Laiko_žymė', 'Įstaigos_pavadinimas'])[inventory_df.columns[8:-3]]\
    .min()\
    .reset_index()\
    .groupby('Laiko_žymė')[inventory_df.columns[8:-3]]\
    .sum()\
    .reset_index()\
    .rename({'Laiko_žymė': 'date'}, axis=1)

inventory_aggregated['beds_taken'] = inventory_aggregated['Užimtos_lovos__vnt_']/inventory_aggregated['Lovos_VISO__vnt_']

In [12]:
df = vilnius_stats_df\
    .merge(personel_aggregated, on='date', how='left')\
    .merge(inventory_aggregated, on='date', how='left')

In [16]:
df.to_clipboard()